<a href="https://colab.research.google.com/github/doc-jones/faiss/blob/main/FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import requests
import os

data_url = "https://raw.githubusercontent.com/jamescalam/data/main/sentence_embeddings_15K/"
s=[]
# create data directory to store data
if not os.path.exists('./data'):
    os.mkdir('./data')

# download the numpy binary files (dense vectors)
for i in range(57):
    if i < 10:
        i = '0' + str(i)
    res = requests.get(data_url+f"embeddings_{i}.npy")

    with open(f'./data/embeddings_{i}.npy', 'wb') as fp:
        for chunk in res:
            fp.write(chunk)

    print('.', end='')

# and download the respective text file
res = requests.get(f"{data_url}sentences.txt")
with open(f"./data/sentences.txt", 'wb') as fp:
    for chunk in res:
        fp.write(chunk)

.........................................................

In [2]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [3]:
import pandas as pd

import requests
from io import StringIO
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [4]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [5]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [6]:
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [7]:


# each of these dataset have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe

    smalldata = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # data=data.append(smalldata,ignore_index=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(smalldata[1].tolist())
    sentences.extend(smalldata[2].tolist())

<ipython-input-7-cee748d90d1e>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  smalldata = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 191: expected 3 fields, saw 4
Skipping line 206: expected 3 fields, saw 4
Skipping line 295: expected 3 fields, saw 4
Skipping line 695: expected 3 fields, saw 4
Skipping line 699: expected 3 fields, saw 4

<ipython-input-7-cee748d90d1e>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  smalldata = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 104: expected 3 fields, saw 4
Skipping line 181: expected 3 fields, saw 4
Skipping line 317: expected 3 fields, saw 4
Skipping line 412: expected 3 fields, saw 5
Skipping line 508: expected 3 fields, saw 4

<ipython-input-7-cee748d90d1

In [ ]:
# smalldata.head()

In [8]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [9]:
len(sentences)

14504

In [10]:
import numpy as np
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.7 MB/s eta 0:00:00


# raw text > into Embeddings [using an encoder] > save it in FAISS > raw text query > into an Embedding > Euclidean distance to retrieve the text closest to the query

In [11]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=4395490dd32f9ef2eb927fdc6c2de30f1fa53253319ad4ecb48a4f1d7de194d7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [12]:
# !git clone https://github.com/jamescalam/data.git data-embeddings

In [13]:
# path = '/content/data-embeddings/sentence_embeddings_15K/'
# sentence_embeddings = []
# for i in range(0,57):
#     # if i < 10:
#     #     i = '0' + str(i)
#     res = path+f"embeddings_{i}.npy"
#     print(res)
#     sm = np.load(res)
#     sentence_embeddings.append(sm)

In [ ]:
# arr = np.concatenate(sentence_embeddings)

In [ ]:
# arr.shape

(14504, 768)

In [14]:
# sentences = sentences[:100]

In [15]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens') #encoding model

In [16]:
len(sentences)

14504

In [17]:
%%time
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

CPU times: user 18.3 s, sys: 1.13 s, total: 19.4 s
Wall time: 24.6 s


(14504, 768)

In [18]:
sentences[0]

'a person under the guardianship or custody of another'

In [19]:
sentence_embeddings[0]

array([ 3.87513250e-01, -2.45500490e-01,  1.71507215e+00,  5.92065871e-01,
        3.47386628e-01,  7.54474699e-01, -6.58478320e-01,  2.46702150e-01,
       -3.66725534e-01, -7.15484321e-02, -3.29468518e-01,  5.98087125e-02,
        2.83082157e-01,  1.26045489e+00, -4.58792835e-01, -1.38534427e-01,
       -3.91573757e-01, -5.71998417e-01,  1.22538768e-01, -1.00487328e+00,
        3.49279165e-01,  3.45602840e-01,  3.62187438e-03, -7.61706531e-01,
       -8.91705811e-01, -2.18302533e-02, -1.66458324e-01, -8.79620731e-01,
       -3.45872670e-01, -1.83460355e-01,  7.57970750e-01, -3.74651670e-01,
        1.30150244e-01,  5.34722097e-02, -1.50511637e-01,  8.13618302e-02,
        8.48397240e-02, -4.81906086e-01,  2.15005159e-01, -5.57236254e-01,
        5.23479164e-01, -3.29723626e-01,  3.80426377e-01,  6.61583245e-01,
       -2.14020193e-01, -7.04577506e-01,  4.71009046e-01,  7.60548785e-02,
       -4.77607638e-01, -7.25326121e-01, -1.12025762e+00, -2.12761298e-01,
        4.83610392e-01,  

In [20]:
# with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
#     np.save(fp, sentence_embeddings[0:256])

In [21]:
# # saving data
# split = 256
# file_count = 0
# for i in range(0, sentence_embeddings.shape[0], split):
#     end = i + split
#     if end > sentence_embeddings.shape[0] + 1:
#         end = sentence_embeddings.shape[0] + 1
#     file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
#     with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
#         np.save(fp, sentence_embeddings[i:end, :])
#     print(f"embeddings_{file_count}.npy | {i} -> {end}")
#     file_count = int(file_count) + 1

In [22]:
import faiss
# sentence_embeddings = arr

In [23]:
d = sentence_embeddings.shape[1]
d

768

In [24]:
sentences[0]

'a person under the guardianship or custody of another'

In [25]:
sentence_embeddings[0].shape

(768,)

In [26]:
index = faiss.IndexFlatL2(d) #euclidean method.  - intialize

In [27]:
index.is_trained

True

In [28]:
index.add(sentence_embeddings)

In [29]:
index.ntotal

14504

converted raw text into embeddings > saved the embeddings into FAISS and we are going to use Euclidean distance to measure the distances

In [30]:
sentences[100:101]

['A black dog running in the snow.']

In [31]:
k = 4
xq = model.encode(["Someone is performing a dance admidst the rainfall."])

Converted the query into an embedding and now we will run Euclidean search on the entire corpus and compare it to the query

In [ ]:
# index

In [32]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[10199  9596 13234  6796]]
CPU times: user 17.3 ms, sys: 0 ns, total: 17.3 ms
Wall time: 17.6 ms


In [33]:
[f'{i}: {sentences[i]}' for i in I[0]]

['10199: A person is dancing in the rain',
 '9596: A man is dancing in the rain',
 '13234: A woman is performing in the rain',
 '6796: The dancer is dancing in front of the sound equipment']

# Quantization

In [34]:
nlist = 100
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [35]:
index.is_trained

False

In [36]:
index.train(sentence_embeddings)
index.is_trained

True

In [37]:
index.add(sentence_embeddings)
index.ntotal

14504

In [38]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 6796  1589 12090  8252]]
CPU times: user 2.04 ms, sys: 0 ns, total: 2.04 ms
Wall time: 2.05 ms


In [39]:
index.nprobe = 10

In [40]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[10199  9596 13234  6796]]
CPU times: user 2.73 ms, sys: 0 ns, total: 2.73 ms
Wall time: 2.79 ms


['582: A person is dancing in the rain',
 '7999: A man is dancing in the rain',
 '14299: A woman is performing in the rain',
 '8088: The dancer is dancing in front of the sound equipment']

In [41]:
[f'{i}: {sentences[i]}' for i in I[0]]

['10199: A person is dancing in the rain',
 '9596: A man is dancing in the rain',
 '13234: A woman is performing in the rain',
 '6796: The dancer is dancing in front of the sound equipment']

In [42]:
index.make_direct_map()

Technique > Time > Subjective performance